# Imports

Dependecies

In [1]:
import os
import qrcode
import numpy as np
import random
from PIL import Image
import cv2
from matplotlib import pyplot as plt

In [2]:
emojis = {
    "check_mark_button":    "\U00002705",
    "cross_mark":           "\U0000274C"
}

# Transparency recovery test

The recovery function takes two transpareces and use them to obtain a qr_code object

## Image load

Generic function to load images (with transparences of distorted transparences)

In [3]:
# 1 - open the images
def load_trans_im(path_im_A="", path_im_B=""):
    """
    """
    if(path_im_A!="" and path_im_B!=""):
        try:
            im_trans_A = Image.open(path_im_A, mode="r")
            im_trans_B = Image.open(path_im_B, mode="r")
            print(emojis["check_mark_button"] + f" Transparence A successfully loaded from {path_im_A}")
            print(emojis["check_mark_button"] + f" Transparence B successfully loaded from {path_im_B}")
            return im_trans_A, im_trans_B
        except:
            print(emojis["cross_mark"] + " Unable to correctly load the image/s")
    else:
        print(emojis["cross_mark"] + f" One or more paths to the image/s were not provided")

In [4]:
!pwd

/Users/eiglesias/Desktop/vcsd/tests


In [5]:
im_trans_A, im_trans_B = load_trans_im(path_im_A='/Users/eiglesias/Desktop/vcsd_test/decryptor/trans_A.png',
                                       path_im_B='/Users/eiglesias/Desktop/vcsd_test/decryptor/trans_B.png')

✅ Transparence A successfully loaded from /Users/eiglesias/Desktop/vcsd_test/decryptor/trans_A.png
✅ Transparence B successfully loaded from /Users/eiglesias/Desktop/vcsd_test/decryptor/trans_B.png


In [6]:
print(type(im_trans_A))
print(type(im_trans_B))

<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>


## Transparence extraction from the loaded images

 Generic function to convert an image of a transparence in a proper numpy array

In [7]:
# 2 - transform images to numpy array format
def gen_transparence_from_image(im):
    """
    """
    trans_rgb = np.array(im)
    trans = trans_rgb.mean(axis=2).astype(int)
    trans = (trans/255).astype(int)
    return trans

In [8]:
print(type(im_trans_A))
print(type(im_trans_B))

<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>


In [9]:
print(im_trans_A.size)

(42, 42)


In [10]:
trans_A = gen_transparence_from_image(im_trans_A)
trans_B = gen_transparence_from_image(im_trans_B)

In [11]:
print("Transparence A")
print(trans_A.shape)
print(trans_A)
print("Transparence B")
print(trans_B.shape)
print(trans_B)

Transparence A
(42, 42)
[[1 1 0 ... 1 0 1]
 [1 1 1 ... 0 1 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 1 0 0]
 [1 0 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]]
Transparence B
(42, 42)
[[0 0 1 ... 0 1 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 1 1]
 ...
 [1 0 0 ... 1 0 0]
 [0 1 1 ... 1 0 1]
 [0 0 0 ... 1 0 0]]


# Transparences decryption test

To be completed

In [12]:
def extract_transparences(trans_A, trans_B, load=False, path_trans_A="", path_trans_B=""):
    """
    """

    if(load):
        # 1 - Call the transparence image load function
        im_trans_A, im_trans_B = load_trans_im(path_trans_A=path_im_A, path_trans_B=path_im_B)   
        # 2 - Call the transparence recover function
        trans_A = gen_transparence_from_image(im_trans_A)
        trans_B = gen_transparence_from_image(im_trans_B)
        
    
    # 3 - check size of transparence to know the correspondent qr code version to generate
    # qr checkversion
    # qr.make
    
    # 4 - extract binary matrix
    
    # 5 - recover QR code matrix applying visual criptography
    
    # 6 - convert that matrix into qr_code object
    # qr.modules = matrix
    
    qr_code_matrix = np.array(qr_code.get_matrix()).astype(int)
    trans_A = np.zeros([x * 2 for x in qr_code_matrix.shape], dtype=qr_code_matrix.dtype)
    trans_B = np.zeros([x * 2 for x in qr_code_matrix.shape], dtype=qr_code_matrix.dtype)

    it = np.nditer(trans_A, flags=['multi_index'])
    submatrix_size = 2

    while not it.finished:
        ri = it.multi_index[0] % submatrix_size
        rj = it.multi_index[1] % submatrix_size

        random_bin = np.reshape(np.random.randint(2, size=submatrix_size**2), (submatrix_size,submatrix_size))

        if ri == 0 and rj == 0:
            start_row    = it.multi_index[0]
            end_row      = it.multi_index[0] + submatrix_size
            start_column = it.multi_index[1]
            end_column   = it.multi_index[1] + submatrix_size
            qr_row    = int(start_row/2)
            qr_column = int(start_column/2)

            if qr_code_matrix[qr_row, qr_column] == 0:
                trans_A[start_row:end_row, start_column:end_column] = random_bin
                trans_B[start_row:end_row, start_column:end_column] = random_bin
            else:
                trans_A[start_row:end_row, start_column:end_column] = random_bin
                random_bin_inv = np.where(random_bin==0, 1, 0)
                trans_B[start_row:end_row, start_column:end_column] = random_bin_inv 

        it.iternext()

    return trans_A, trans_B